In [2]:
import os
import time
import multiprocessing as mp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import func4behav as fb
import itertools
import ast
import imp
imp.reload(fb)

<module 'func4behav' from 'c:\\Users\\Tianyi Zheng\\OneDrive - neuron.t.u-tokyo.ac.jp\\Documents\\zheng\\mywork\\attention_tES\\tes-attention\\processing\\behavior\\func4behav.py'>

In [39]:
def auto_sig(size, watch_cases, watch_idxs):
    record = pd.DataFrame(columns=['try id', 'Real picks', 'Sham picks', 'p values', 'Judge'])
    all_real = [1, 3, 4, 5, 9, 12, 13, 17, 18]
    all_sham = [2, 6, 7, 8, 10, 11, 14, 15, 16]
    real_comb = [list(comb) for comb in itertools.combinations(all_real, size)]
    sham_comb = [list(comb) for comb in itertools.combinations(all_sham, size)]

    start_time = time.time()
    try_id = 0

    for real_to_pick in real_comb:
        for sham_to_pick in sham_comb:
            p_values = fb.auto_compare(real_to_pick, sham_to_pick, watch_cases, watch_idxs)
            record.at[try_id, 'try id'] = try_id
            record.at[try_id, 'Real picks'] = real_to_pick
            record.at[try_id, 'Sham picks'] = sham_to_pick
            record.at[try_id, 'p values'] = p_values.values.tolist()
            record.at[try_id, 'Judge'] = [1 if all(i <= 0.05 for i in sublist if i is not None) else 0 for sublist in p_values.values]

            judge_matrix = p_values.applymap(lambda x: 1 if x <= 0.05 else 0)
            record.at[try_id, 'Judge'] = judge_matrix.values.tolist()

            try_id += 1

    end_time = time.time()
    elapsed_time = end_time - start_time  # Calculate the difference
    print(f"The program took {elapsed_time} seconds to run.")
    record.to_csv('../../../docs/report/data/record_'+str(size)+'.csv', index=False)

    return elapsed_time

In [40]:
watch_cases = ['endo', 'exo', 'endo valid', 'exo valid', 'endo invalid', 'exo invalid', 'endo stim left', 'endo stim right', 'exo stim left', 'exo stim right']
watch_idxs = ["RT mean shorten", "RT median shorten", "RT mean shorten %", "RT median shorten %"]
size = 9
auto_sig(size, watch_cases, watch_idxs)

The program took 0.8789994716644287 seconds to run.


0.8789994716644287

### View result

In [3]:
def round_nested_lists(nested_list):
    return [[round(number, 3) for number in sublist] for sublist in nested_list]

# Define a function to sum the sublist elements column-wise
def sum_sublist_elements(lst):
    return [sum(x) for x in zip(*lst)]

In [7]:
record_path = os.path.join('..', '..', '..', 'docs', 'report', 'data', 'record_6'+'.csv')
record = pd.read_csv(record_path)

record['p values'] = record['p values'].apply(ast.literal_eval).apply(round_nested_lists)
# Convert the 'Judge' column from string representation to actual list of lists
record['Judge'] = record['Judge'].apply(ast.literal_eval)

# Filter the dataframe
record = record[record['Judge'].apply(lambda row: any(1 in sublist for sublist in row))]


# Apply this function to the 'Judge' column and create a new column 'Judge_sum'
record['Judge sum'] = record['Judge'].apply(sum_sublist_elements)

# Create a new column that contains the maximum value from the 'Judge_sum' list
record['Max Judge sum'] = record['Judge sum'].apply(max)

# Sort the DataFrame by the 'Max_Judge_sum' column in descending order
record = record.sort_values(by='Max Judge sum', ascending=False)

record.drop(columns=['Max Judge sum'], inplace=True)

record


,try id,Real picks,Sham picks,p values,Judge,Judge sum
2132,2132,"[1, 3, 5, 9, 17, 18]","[2, 6, 10, 11, 15, 16]","[[0.065, 0.002, 0.015, 0.002], [0.132, 0.132, ...","[[0, 1, 1, 1], [0, 0, 0, 0], [0, 1, 0, 1], [0,...","[1, 4, 2, 4]"
2130,2130,"[1, 3, 5, 9, 17, 18]","[2, 6, 10, 11, 14, 15]","[[0.18, 0.009, 0.093, 0.009], [0.394, 0.394, 0...","[[0, 1, 0, 1], [0, 0, 0, 0], [0, 1, 0, 1], [0,...","[1, 4, 1, 4]"
2111,2111,"[1, 3, 5, 9, 17, 18]","[2, 6, 7, 10, 11, 15]","[[0.026, 0.009, 0.015, 0.004], [0.132, 0.31, 0...","[[1, 1, 1, 1], [0, 0, 0, 0], [0, 1, 0, 1], [0,...","[2, 4, 2, 4]"
6930,6930,"[4, 9, 12, 13, 17, 18]","[2, 7, 8, 11, 14, 16]","[[0.24, 0.485, 0.18, 0.485], [0.093, 0.132, 0....","[[0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 0], [1,...","[1, 0, 4, 0]"
2110,2110,"[1, 3, 5, 9, 17, 18]","[2, 6, 7, 10, 11, 14]","[[0.18, 0.026, 0.093, 0.015], [0.394, 0.394, 0...","[[0, 1, 0, 1], [0, 0, 0, 0], [0, 1, 0, 0], [0,...","[1, 3, 1, 2]"
...,...,...,...,...,...,...
2143,2143,"[1, 3, 5, 9, 17, 18]","[2, 7, 8, 11, 15, 16]","[[0.31, 0.093, 0.132, 0.093], [0.485, 0.699, 0...","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[0, 1, 0, 0]"
2146,2146,"[1, 3, 5, 9, 17, 18]","[2, 7, 10, 11, 14, 16]","[[0.31, 0.026, 0.093, 0.015], [0.589, 0.485, 0...","[[0, 1, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[0, 1, 0, 1]"
2148,2148,"[1, 3, 5, 9, 17, 18]","[2, 7, 10, 14, 15, 16]","[[0.31, 0.026, 0.093, 0.015], [0.699, 0.937, 0...","[[0, 1, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[0, 1, 0, 1]"
2149,2149,"[1, 3, 5, 9, 17, 18]","[2, 7, 11, 14, 15, 16]","[[0.31, 0.026, 0.093, 0.015], [0.818, 0.937, 0...","[[0, 1, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[0, 1, 0, 1]"


In [22]:
display(record.at[2132, 'Judge'])
display(record.at[2132, 'Judge sum'])

[[0, 1, 1, 1],
 [0, 0, 0, 0],
 [0, 1, 0, 1],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 1, 0, 1],
 [0, 0, 0, 0],
 [1, 1, 1, 1]]

[1, 4, 2, 4]